# Sanity Rollouts — Random vs Tiny Heuristic (GGEnv v2)

**Goal.** Validate that our environment and observations are sane by comparing:
- a **Random** policy (uniform over {NOOP, FLIP})
- a **Tiny Heuristic** (flip only when near-lane is immediately dangerous and target lane looks safe)

We:
1. (Optionally) run rollouts to generate fresh results.
2. Load `experiments/runs/episodes.csv`.
3. Check basic data integrity.
4. Compare metrics: distance, episode length, death causes.
5. Do a seed-paired comparison (does heuristic consistently beat random?).
6. Summarize findings and next steps.

*Environment:* GGEnv v2, Observation v2 (15-dim), frame_skip=4 by default.


In [ ]:
import os, sys, math
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Paths
PROJECT_ROOT = Path.cwd()  # run notebook from repo root if possible
RUNS_DIR = PROJECT_ROOT / "experiments" / "runs"
EPISODES_CSV = PROJECT_ROOT / "episodes.csv"

print("Project root:", PROJECT_ROOT)
print("Runs dir:", RUNS_DIR)
print("Episodes CSV exists:", EPISODES_CSV.exists())

# Matplotlib defaults
plt.rcParams["figure.figsize"] = (8, 4.5)
plt.rcParams["axes.grid"] = True
plt.rcParams["axes.spines.top"] = False
plt.rcParams["axes.spines.right"] = False


Project root: d:\Projects\GravityGuyML\experiments\notebooks
Runs dir: d:\Projects\GravityGuyML\experiments\notebooks\experiments\runs
Episodes CSV exists: False
